In [1]:
import pytesseract

In [7]:
image = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/0632598000/cells/003-001.png'
pytesseract.image_to_string(
                image,
                lang='eng', config='psm--6'
            )

'118,633 61—\n\x0c'

In [8]:
import pytesseract
image = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/0105253000/cells/002-001.png'
pytesseract.image_to_string(
                image,
                lang='eng', config='psm--6'
            )

'17,302 96\n\x0c'

In [9]:
image = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/0423173000/cells/003-001.png'
pytesseract.image_to_string(
                image,
                lang='eng', config='psm--6'
            )

'149,631.18\n\x0c'

In [10]:
image = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/0319768000/cells/003-001.png'
pytesseract.image_to_string(
                image,
                lang='eng', config='psm--6'
            )

'38,519.59\n\x0c'

In [2]:
import pytesseract
from PIL import Image
r = Image.open('/home/vimalkumar.s/Documents/file samples/test/1st page/0105253000.jpeg')
r.load()
#Converting inmage to text with preserving interline spaces
text = pytesseract.image_to_string(r,config='--psm 11' )
text

'MISSOURI DIV OF EMPLOYMENT SECURITY\n\ne\n\nUNEMPLOYMENT INSURANCE TAX\n\n373-751-1995\n\nQUARTERLY CONTRIBUTION\n\n2 MO EMPLOYER ACCOUNT NO\n\nYEAR\n\nAUDIT\n\nAND WAGE REPORT\n\n0105253000\n\n2021\n\n(DO NOT\n\nFile online at umteract.labor.mo.gov\n\nEAU4\n\nUSE)\n\n3 CALENDAR QUARTER\n\nDate Paid\n\n1 EMPLOYER NAME AND ADDRESS\n\nIst\n\nx\n\n2nd\n\n3rd\n\n4th\n\nGentry County Senior Center Inc\n\n219 North High Street\n\nMUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO\n\n4 TOTAL WAGES PAID\n\nStanberry\n\nMO\n\n64489\n\n17,302 96\n\n5 WAGES PAID IN EXCESS OF\n\n11,000 00 PER WORKER\n\n0 00\n\nPER YEAR (See Instruction Sheet)\n\n6 TAXABLE WAGES\n\n(Item 4 Minus Item 5)\n\n17,302 96\n\n7 TAXES DUE (Multiply Item 6\n\nby Your Rate)\n\n0 0000\n\n0 00\n\n14 FEDERAL ID NUMBER 43-1092074\n\n8 INTEREST ASSESSMENT DUE\n\nTO FEDERAL ADVANCES\n\nIf mailing, return this page with remittance to\n\nDivision of Employment Security\n\n9 INTEREST CHARGES OF\n\nP O. Box 888\n\nPER MONTH IF\n\nPAID AFTE

In [3]:
import cv2
import os

def extract_cell_images_from_table(image):
    BLUR_KERNEL_SIZE = (17, 17)
    STD_DEV_X_DIRECTION = 0
    STD_DEV_Y_DIRECTION = 0
    blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2
    
    img_bin = cv2.adaptiveThreshold(
        ~blurred,
        MAX_COLOR_VAL,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        BLOCK_SIZE,
        SUBTRACT_FROM_MEAN,
    )
    vertical = horizontal = img_bin.copy()
    SCALE = 5
    image_width, image_height = horizontal.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
    horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
    vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    
    horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
    vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))
    
    mask = horizontally_dilated + vertically_dilated
    contours, heirarchy = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
    )
    
    perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
    epsilons = [0.05 * p for p in perimeter_lengths]
    approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]
    
    # Filter out contours that aren't rectangular. Those that aren't rectangular
    # are probably noise.
    approx_rects = [p for p in approx_polys if len(p) == 4]
    bounding_rects = [cv2.boundingRect(a) for a in approx_polys]
    
    # Filter out rectangles that are too narrow or too short.
    MIN_RECT_WIDTH = 40
    MIN_RECT_HEIGHT = 10
    bounding_rects = [
        r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
    ]
    
    # The largest bounding rectangle is assumed to be the entire table.
    # Remove it from the list. We don't want to accidentally try to OCR
    # the entire table.
    largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
    bounding_rects = [b for b in bounding_rects if b is not largest_rect]
    
    cells = [c for c in bounding_rects]
    def cell_in_same_row(c1, c2):
        c1_center = c1[1] + c1[3] - c1[3] / 2
        c2_bottom = c2[1] + c2[3]
        c2_top = c2[1]
        return c2_top < c1_center < c2_bottom
    
    orig_cells = [c for c in cells]
    rows = []
    while cells:
        first = cells[0]
        rest = cells[1:]
        cells_in_same_row = sorted(
            [
                c for c in rest
                if cell_in_same_row(c, first)
            ],
            key=lambda c: c[0]
        )
    
        row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
        rows.append(row_cells)
        cells = [
            c for c in rest
            if not cell_in_same_row(c, first)
        ]
    
    # Sort rows by average height of their center.
    def avg_height_of_center(row):
        centers = [y + h - h / 2 for x, y, w, h in row]
        return sum(centers) / len(centers)
    
    rows.sort(key=avg_height_of_center)
    cell_images_rows = []
    for row in rows:
        cell_images_row = []
        for x, y, w, h in row:
            cell_images_row.append(image[y:y+h, x:x+w])
        cell_images_rows.append(cell_images_row)
    return cell_images_rows

def main(f):
    results = []
    directory, filename = os.path.split(f)
    table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    rows = extract_cell_images_from_table(table)
    cell_img_dir = os.path.join(directory, "cells")
    os.makedirs(cell_img_dir, exist_ok=True)
    paths = []
    for i, row in enumerate(rows):
        for j, cell in enumerate(row):
            cell_filename = "{:03d}-{:03d}.png".format(i, j)
            path = os.path.join(cell_img_dir, cell_filename)
            cv2.imwrite(path, cell)
            paths.append(path)
    return paths

In [4]:
f = '/home/vimalkumar.s/Documents/file samples/test/1st page/0102170000.jpeg'
main(f)

['/home/vimalkumar.s/Documents/file samples/test/1st page/cells/000-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/001-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/002-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/002-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/004-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/004-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/005-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/005-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/006-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/006-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/

In [7]:
text = []
path = '/home/vimalkumar.s/Documents/file samples/test/1st page/cells'
file_path = sorted(os.listdir(path))
for x in file_path:
    txt = pytesseract.image_to_string(os.path.join(path,x))
    text.append(txt)
print(text)

['2 MO EMPLOYER ACCOUNT NO\n\nYEAR\n\n \n\n \n\n \n\n01-02170-0-00 2021\n3 CALENDAR QUARTER\nIst [| yx | 2nd 3rd 4th\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\x0c', 'MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO\n\x0c', '4 TOTAL WAGES PAID\n\x0c', '246,887 30\n\x0c', '5 WAGES PAID IN EXCESS OF\n11,000.00 PER WORKER\nPER YEAR (See Instruction Sheet)\n\x0c', '14,075 92\n\x0c', '6 TAXABLE WAGES\n(Itern 4 Minus Item 5)\n\x0c', '232,811 38\n\x0c', '7 TAXES DUE (Multply Item 6\nby Your Rate) 1.4080\n\x0c', '3,277 98\n\x0c', '8 INTEREST ASSESSMENT DUE\nTO FEDERAL ADVANCES\n\x0c', ' \n\n \n\n \n\n \n\n2 MOEMPLOYER ACCOUNT NO | YEAR AUDIT\n\n(DONOT\n01-02170-0-00 2021 USE)\n3 CALENDAR QUARTER Date Pad\nIst | yx | 2nd 3rd 4th\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nMUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO\n\n \n\n4 TOTAL WAGES PAID\n\n246,887\n\n30\n\n \n\n5 WAGES PAID IN EXCESS OF\n11,000.00 PER WORKER\nPER YEAR (See Instruction Sheet)\n\n14,075\n\n92\n\n \n\n6 TA

In [8]:
for i in text:
    print(i)

2 MO EMPLOYER ACCOUNT NO

YEAR

 

 

 

01-02170-0-00 2021
3 CALENDAR QUARTER
Ist [| yx | 2nd 3rd 4th

 

 

 

 

 

 

 

 

 

MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO

4 TOTAL WAGES PAID

246,887 30

5 WAGES PAID IN EXCESS OF
11,000.00 PER WORKER
PER YEAR (See Instruction Sheet)

14,075 92

6 TAXABLE WAGES
(Itern 4 Minus Item 5)

232,811 38

7 TAXES DUE (Multply Item 6
by Your Rate) 1.4080

3,277 98

8 INTEREST ASSESSMENT DUE
TO FEDERAL ADVANCES

 

 

 

 

2 MOEMPLOYER ACCOUNT NO | YEAR AUDIT

(DONOT
01-02170-0-00 2021 USE)
3 CALENDAR QUARTER Date Pad
Ist | yx | 2nd 3rd 4th

 

 

 

 

 

 

 

 

 

 

 

 

MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO

 

4 TOTAL WAGES PAID

246,887

30

 

5 WAGES PAID IN EXCESS OF
11,000.00 PER WORKER
PER YEAR (See Instruction Sheet)

14,075

92

 

6 TAXABLE WAGES
(Item 4 Minus Item 5)

232,811

38

 

7 TAXES DUE (Multiply Item 6
by Your Rate) 1.4080

3,277

98

 

8 INTEREST ASSESSMENT DUE
TO FEDERAL ADVANCES

 

9 INTERES

In [9]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/home/vimalkumar.s/Documents/python_new/cde_venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [17]:
word = 'TOTAL WAGES PAID'
for i in text:
    if fuzz.ratio(word,i) > 80:
        print(i)
        print(text.index(i))
        ind = text.index(i)
        out = text[ind+1]
        print(out)
        

4 TOTAL WAGES PAID

2
246,887 30



In [18]:
import cv2
import os

def extract_cell_images_from_table(image):
    BLUR_KERNEL_SIZE = (17, 17)
    STD_DEV_X_DIRECTION = 0
    STD_DEV_Y_DIRECTION = 0
    blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2
    
    img_bin = cv2.adaptiveThreshold(
    ~blurred,
    MAX_COLOR_VAL,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY,
    BLOCK_SIZE,
    SUBTRACT_FROM_MEAN,
    )
    vertical = horizontal = img_bin.copy()
    SCALE = 5
    image_width, image_height = horizontal.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
    horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
    vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
    vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

    mask = horizontally_dilated + vertically_dilated
    contours, heirarchy = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
    )

    perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
    epsilons = [0.05 * p for p in perimeter_lengths]
    approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

    # Filter out contours that aren't rectangular. Those that aren't rectangular
    # are probably noise.
    approx_rects = [p for p in approx_polys if len(p) == 4]
    bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

    # Filter out rectangles that are too narrow or too short.
    MIN_RECT_WIDTH = 40
    MIN_RECT_HEIGHT = 10
    bounding_rects = [
        r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
    ]
    
    if len(bounding_rects)==0:
        print('00')
    else:
        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows
def main(f):
    results = []
    directory, filename = os.path.split(f)
    table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    rows = extract_cell_images_from_table(table)
    print(rows)
    print(type(rows))
    cell_img_dir = os.path.join(directory, "cells")
    os.makedirs(cell_img_dir, exist_ok=True)
    paths = []
    for i, row in enumerate(rows):
        for j, cell in enumerate(row):
            cell_filename = "{:03d}-{:03d}.png".format(i, j)
            path = os.path.join(cell_img_dir, cell_filename)
            cv2.imwrite(path, cell)
            paths.append(path)
    return paths
f = '/home/vimalkumar.s/Documents/file samples/test/1st page/0905808000.jpeg'
main(f)

[[array([[255, 255, 249, ..., 247,   0,   2],
       [255, 249, 255, ..., 253,   0,   0],
       [255, 249, 255, ..., 251,   0,   1],
       ...,
       [252, 255, 255, ..., 244, 255,   0],
       [250, 252, 245, ..., 255, 239,   6],
       [255, 255, 255, ..., 255,   9,   4]], dtype=uint8)], [array([[  0,   0,   4, ..., 253, 255, 252],
       [255, 255, 252, ..., 253, 255, 252],
       [255, 253, 255, ..., 253, 255, 252],
       ...,
       [237, 255, 248, ..., 255, 252, 223],
       [255, 255, 255, ..., 235, 255, 255],
       [245, 254, 238, ...,  15,   0,  16]], dtype=uint8), array([[254, 255, 250, ..., 252, 251,   0],
       [255, 246, 255, ..., 255, 255,   8],
       [248, 255, 247, ..., 244, 255,   0],
       ...,
       [255, 231, 249, ...,   0,   0,   2],
       [253, 255, 255, ...,  10,   6,   0],
       [  8,   0,   0, ..., 247, 241,   9]], dtype=uint8)], [array([[254, 238, 255, ...,  15,   0,  16],
       [ 17,  14,   0, ..., 251, 255, 253],
       [246, 255,   0, ..., 245, 

['/home/vimalkumar.s/Documents/file samples/test/1st page/cells/000-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/001-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/001-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/002-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/002-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-002.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/004-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/004-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/005-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/005-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/

In [19]:
import re
import pytesseract
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

text = []
path = '/home/vimalkumar.s/Documents/file samples/test/1st page/cells'
file_path = sorted(os.listdir(path))
for x in file_path:
    txt = pytesseract.image_to_string(os.path.join(path,x))
    text.append(txt)
print(text)
word = 'TOTAL WAGES PAID'
a = []
for i in text:
    if fuzz.ratio(word,i) > 75:
        print(i)
        a.append(i)
        print(text.index(i))
        ind = text.index(i)
        out = text[ind+1]
        print(out)
        num_1 = re.sub("[ ,.#*!@$%^&()|\''"":;-]", "", out)
        print(num_1)
        

['MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO\n\x0c', '\x0c', '4 TOTAL WAGES PAID\n\x0c', '9,802 13\n\x0c', '5 WAGES PAID IN EXCESS OF\n11,000 00 PER WORKER\n\n \n\x0c', '\x0c', '6 TAXABLE WAGES\n(Item 4 Minus Item 5)\n\n7 TAXES DUE (Muluply Item 6\nby Your Rate) 2 5110\n\x0c', '9,802 13\n\n246 13\n\n \n\x0c', 'AUDIT\n{DO NOT\nUSE)\nDate Paid\n\n \n\n \n\x0c', '8 INTEREST ASSESSMENT DUE -\nTO FEDERAL ADVANCES\n\x0c', '\x0c', '\x0c', 'If masling, return this page with remittance to\nDivision of Employment Security\nP.O. Box 888\nJefferson City, MO 65102-0888\nMake check payable to Division of Employment Security\nor pay online at uinteract labor mo gov\n\n   \n \n  \n\x0c', '9 INTEREST CHARGES OF\nPER MONTH IF\nPAID AFTER 04/30/21\n\x0c', '\x0c', '10 LATE REPORT PENALTY\nCHARGES (See Item 15 to the Left)\n\x0c', '\x0c', '\x0c', '15 THIS REPORT IS DUE BY 04/30/21\nGREATER OF 10% OR $100 PENALTY AFTER 05/31/21\n"GREATER OF 20% OR $200 PENALTY AFTER 06/30/21\n\n \n\x0c', '11 QUTSTANDING A

In [21]:
word = 'TOTAL WAGES PAID'
a = []
for i in text:
    if fuzz.ratio(word,i) > 77:
        print(i)
        a.append(i)
        print(text.index(i))
        ind = text.index(i)
        out = text[ind+1]
        print(out)
        num_1 = re.sub("[ ,.#*!@$%^&()|\''"":;-_]", "", out)
        print(num_1)
        op = list(num_1)
if op[0].isdigit():
    print('TWP',num_1)
else:
    print('00')
        
if len(a) == 0:
    print('000')

4 TOTAL WAGES PAID

2
9,802 13

980213

TWP 980213



In [41]:
word = 'TOTAL WAGES PAID'
a = []
for i in text:
    if fuzz.ratio(word,i) > 77:
        print(i)
        a.append(i)
        if len(a) != 0:
            print(text.index(i))
            ind = text.index(i)
            out = text[ind+1]
            print(out)
            num_1 = re.sub("[ ,.#*!@$%^&()|\''"":;-]", "", out)
            print(num_1)
else:
    print('000')
        

000


In [5]:
out

NameError: name 'out' is not defined

In [75]:
import re
num_1 = re.sub("[ ,.#*!@$%^&()|\''"":;-]", "", out)
num_1

'87000\n\x0c'

In [ ]:
0249097000
0365088000
0964707000
0557316000
0107753000
0651221000

In [26]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import cv2
import os
import math
import sys
import numpy as np
import pytesseract
import io 
import csv
import pandas as pd
import shutil, os
import re
import warnings
warnings.filterwarnings('ignore')
path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/input'
for file_name in os.listdir(path):
    
    #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                     os.path.splitext(file_name)[0]))
    
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                    os.path.splitext(file_name)[0])
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths
    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/input',
                               os.path.splitext(file_name)[0]+'.tif')
    main(f)
    
    text = []
    path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                    os.path.splitext(file_name)[0])
    file_path = sorted(os.listdir(path))
    for x in file_path:
        txt = pytesseract.image_to_string(os.path.join(path,x))
        text.append(txt)
    #print(text)
    word = 'TOTAL WAGES PAID'
    a = []
    for i in text:
        if fuzz.ratio(word,i) > 77:
            #print(i)
            a.append(i)
            #print(text.index(i))
            ind = text.index(i)
            out = text[ind+1]
            #print(out)
            num_1 = re.sub("[ ,.#*!@$%^&()|\''"":;-_+]", "", out)
            #print(num_1)
            op = list(num_1)
            if op[0].isdigit():
                print(file_name,'TOTAL WAGES PAID',num_1)
            else:
                print(file_name,'TOTAL WAGES PAID','00')
        
    if len(a) == 0:
        print(file_name,'TOTAL WAGES PAID','000')
    


0450140000.tif TOTAL WAGES PAID 000
0905808000.tif TOTAL WAGES PAID 980213

0462378000.tif TOTAL WAGES PAID 6333830

0911520000.tif TOTAL WAGES PAID 3954989

0313471000.tif TOTAL WAGES PAID 1466625

0356369000.tif TOTAL WAGES PAID 4193025

0300168000.tif TOTAL WAGES PAID 000
0771062000.tif TOTAL WAGES PAID 876201

0560522000.tif TOTAL WAGES PAID 28359226

0308511000.tif TOTAL WAGES PAID 1267500

0719339000.tif TOTAL WAGES PAID 00
0954699000.tif TOTAL WAGES PAID 174250

0632871000.tif TOTAL WAGES PAID 7926648

0241904000.tif TOTAL WAGES PAID 000
0327919000.tif TOTAL WAGES PAID 372500

0809963000.tif TOTAL WAGES PAID 3966315

0352450000.tif TOTAL WAGES PAID 000
0878579000.tif TOTAL WAGES PAID 88000

0602441000.tif TOTAL WAGES PAID 5

eenstructionheet

0489279000.tif TOTAL WAGES PAID 1096814

0644730000.tif TOTAL WAGES PAID 2281796

0222259000.tif TOTAL WAGES PAID 789494

0902476000.tif TOTAL WAGES PAID 11790266

0784699000.tif TOTAL WAGES PAID 000
0963660000.tif TOTAL W

0233728000.tif TOTAL WAGES PAID 000
0319768000.tif TOTAL WAGES PAID 3851959

0655193000.tif TOTAL WAGES PAID 000
0137136000.tif TOTAL WAGES PAID 00
0343681000.tif TOTAL WAGES PAID 000
0744472000.tif TOTAL WAGES PAID 00
0316327000.tif TOTAL WAGES PAID 120800

0587994000.tif TOTAL WAGES PAID 000
0805989000.tif TOTAL WAGES PAID 250000

0327815000.tif TOTAL WAGES PAID 1658800

0499887000.tif TOTAL WAGES PAID 2462625

0412027000.tif TOTAL WAGES PAID 000
0581672000.tif TOTAL WAGES PAID 140001

0879393000.tif TOTAL WAGES PAID 750025

0891827000.tif TOTAL WAGES PAID 309537

0742352000.tif TOTAL WAGES PAID 9751333

0923203000.tif TOTAL WAGES PAID 17419479

0950474000.tif TOTAL WAGES PAID 324000

0645812000.tif TOTAL WAGES PAID 4088991

0458883000.tif TOTAL WAGES PAID 1844384

0674115000.tif TOTAL WAGES PAID 000
0372748000.tif TOTAL WAGES PAID 23911592

0214267000.tif TOTAL WAGES PAID 000
0950006000.tif TOTAL WAGES PAID 00
0288014000.tif TOTAL WAGES PAID 3078103

0849127000.tif TO

0333322000.tif TOTAL WAGES PAID 000
0216226000.tif TOTAL WAGES PAID 5650845

0573626000.tif TOTAL WAGES PAID 000
0841493000.tif TOTAL WAGES PAID 1920000

0612066000.tif TOTAL WAGES PAID 000
0358542000.tif TOTAL WAGES PAID 4166759

0912299000.tif TOTAL WAGES PAID 499200

0633979000.tif TOTAL WAGES PAID 3595576

0746730000.tif TOTAL WAGES PAID 21532143

0593002000.tif TOTAL WAGES PAID 705210

0308610000.tif TOTAL WAGES PAID 5829000

0944759000.tif TOTAL WAGES PAID 384725

0558226000.tif TOTAL WAGES PAID 600000

0692253000.tif TOTAL WAGES PAID 12154950

0400462000.tif TOTAL WAGES PAID 000
0928685000.tif TOTAL WAGES PAID 1000000

0367685000.tif TOTAL WAGES PAID 000
0667314000.tif TOTAL WAGES PAID 673076

0154722000.tif TOTAL WAGES PAID 000
0943362000.tif TOTAL WAGES PAID 4078454

0469981000.tif TOTAL WAGES PAID 00
0737994000.tif TOTAL WAGES PAID 6956137

0753838000.tif TOTAL WAGES PAID 000
0796110000.tif TOTAL WAGES PAID 577500

0870857000.tif TOTAL WAGES PAID 17904181

04

In [22]:
str1 = '4 TOTAL WAGES PAID 4386'
str2 = 'TOTAL WAGES PAID'
fuzz.ratio(str2,str1)

82

In [11]:
a = ['25aa436']

In [12]:
if a[0].isdigit():
    print('true')
else:
    print('false')

false
